In [2]:
import pandas as pd
import numpy as np
import requests
import time

In [3]:
cities = pd.read_csv(r'C:\Users\nnoar\OneDrive\Desktop\homework\Project_3_Weather\cities.csv')
cities.head()

,rank,name,usps,pop2021,pop2010,growth,density,aland_sqmi
0,1,New York City,NY,8230290,8190210,0.0049,27400,300.381
1,2,Los Angeles,CA,3983540,3795510,0.0495,8494,468.956
2,3,Chicago,IL,2679080,2697480,-0.0068,11783,227.369
3,4,Houston,TX,2323660,2100280,0.1064,3630,640.194
4,5,Phoenix,AZ,1733630,1449040,0.1964,3349,517.673


In [4]:
city_list = []
for city in cities['name']:
    city_list.append(city)
    
    

In [5]:
url = 'http://api.openweathermap.org/data/2.5/weather?'
units = 'metric'
query_url = f'{url}appid={api_key}&units={units}&q='

city_name = []
lat = []
lng = []
max_temp = []
humidity = []
cloudiness = []
wind_speed = []
country = []
date = []


for city in city_list:
        response = requests.get(query_url + city).json()
        city_name.append(city)
        lat.append(response['coord']['lat'])
        lng.append(response['coord']['lon'])
        max_temp.append(response['main']['temp_max'])
        humidity.append(response['main']['humidity'])
        wind_speed.append(response['wind']['speed'])
        country.append(response['sys']['country'])
        cloudiness.append(response['clouds']['all'])
        date.append(response['dt'])


In [6]:
weather = {'City':city_name,'Lat':lat,'Lng':lng,'Max Temp':max_temp,'Humidity':humidity,'Cloudiness':cloudiness,'Wind Speed':wind_speed,'Country':country,'Date':date}
weather_df = pd.DataFrame({ key:pd.Series(value) for key, value in weather.items() })



In [7]:
weather_json = weather_df.to_json(orient='index')

In [8]:
weather_dict = weather_df.to_dict()

In [9]:
weather_dict.keys()

dict_keys(['City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed', 'Country', 'Date'])

In [10]:
import json
with open('weather_json', 'w') as fp:
    json.dump(weather_json, fp)

In [12]:
weather_df.to_csv('weather_df.csv')

In [35]:
url = 'http://api.openweathermap.org/data/2.5/forecast?'
units = 'metric'
api_key = ''
query_url = f'{url}appid={api_key}&units={units}&q='


city_name = []
lat_5 = []
lng_5 = []
max_temp_5 = []
humidity_5 = []
cloudiness_5 = []
wind_speed_5 = []
date_5 = []


for city in city_list:
        for i in range(0,40,8):
                response = requests.get(query_url + city).json()
                city_name.append(city)
                lat_5.append(response['city']['coord']['lat'])
                lng_5.append(response['city']['coord']['lon'])
                max_temp_5.append(response['list'][i]['main']['temp_max'])
                humidity_5.append(response['list'][i]['main']['humidity'])
                wind_speed_5.append(response['list'][i]['wind']['speed'])
                cloudiness_5.append(response['list'][i]['clouds']['all'])
                date_5.append(response['list'][i]['dt_txt'])


In [36]:
weather_5 = {'City':city_name,'Lat':lat_5,'Lng':lng_5,'Max Temp':max_temp_5,'Humidity':humidity_5,'Cloudiness':cloudiness_5,'Wind Speed':wind_speed_5,'Date':date_5}
weather5_df = pd.DataFrame({ key:pd.Series(value) for key, value in weather_5.items() })



In [37]:
weather5_df.head(20)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,New York City,40.7143,-74.0060,13.75,65,53,8.93,2021-11-13 21:00:00
1,New York City,40.7143,-74.0060,8.11,47,75,4.57,2021-11-14 21:00:00
2,New York City,40.7143,-74.0060,8.30,50,81,7.36,2021-11-15 21:00:00
3,New York City,40.7143,-74.0060,9.22,41,9,5.66,2021-11-16 21:00:00
4,New York City,40.7143,-74.0060,12.69,64,100,2.45,2021-11-17 21:00:00
5,Los Angeles,34.0522,-118.2437,30.18,18,1,1.91,2021-11-13 21:00:00
6,Los Angeles,34.0522,-118.2437,30.08,13,0,2.64,2021-11-14 21:00:00
7,Los Angeles,34.0522,-118.2437,28.04,17,6,2.41,2021-11-15 21:00:00
8,Los Angeles,34.0522,-118.2437,23.08,33,100,3.15,2021-11-16 21:00:00
9,Los Angeles,34.0522,-118.2437,22.96,39,5,2.49,2021-11-17 21:00:00


In [38]:
weather5_df.to_csv('weather_5.csv')